In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

In [2]:
import pandas as pd
df = pd.read_csv("gutenberg_ebooks.csv")

In [3]:
import pandas as pd

# only removes funny tokens for English texts

def remove_funny_tokens(text):
    tokens = text.split()
    sample = ' '.join(' '.join(tokens).replace('xe2x80x9c', ' ').replace('xe2x80x9d', ' ') \
                                      .replace('xe2x80x94', ' ').replace('xe2x80x99', "'") \
                                      .replace('xe2x80x98', "'").split())
    return sample
# clean newlines, carriage returns and tabs
def clean_text(text):
    if not isinstance(text, str):  # Vérifiez si le texte est une chaîne
        return text  # Retournez la valeur d'origine si ce n'est pas une chaîne
    
    cleaned_listed_text = []
    listed_text = list(text)

    for iter in range(len(listed_text) - 1):
        if (listed_text[iter] == '\\' and listed_text[iter + 1] == 'n') or \
            (listed_text[iter] == 'n' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\' and listed_text[iter + 1] == 'r' or \
            (listed_text[iter] == 'r' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\' and listed_text[iter + 1] == 't' or \
            (listed_text[iter] == 't' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\':
            continue
        else:
            cleaned_listed_text.append(listed_text[iter])

    cleaned_text = ''.join([str(char) for char in cleaned_listed_text])
    cleaned_text = remove_funny_tokens(cleaned_text)

    return ''.join(cleaned_text)

# Charger le CSV déjà existant
df_data = pd.read_csv('gutenberg_ebooks.csv')

# Nettoyer le texte dans la colonne 'Summary'
df_data['Summary'] = df_data['Summary'].apply(clean_text)

# Sauvegarder les données nettoyées dans un nouveau CSV
df_data.to_csv('gutenberg_cleaned_ebooks.csv', index=False)

In [4]:

df = pd.read_csv("gutenberg_cleaned_ebooks.csv")
df

,Ebook ID,Author,Title,Credits,Summary,Language,LoC Class,Subject,Subject_2,Subject_3,Subject_4,Category,EBook-No.,Release Date,Most Recently Updated,Copyright Status,Downloads
0,8600,"Zola, Émile, 1840-1902",L'Assommoir,"John Bickers, Dagny and David Widger","""L'Assommoir"" by Émile Zola is a novel written...",English,PQ: Language and Literatures: Romance literatu...,Domestic fiction,Married women -- Fiction,Paris (France) -- Fiction,Working class women -- Fiction,Text,8600,"Apr 27, 2006","Sep 15, 2022",Public domain in the USA.,455 downloads in the last 30 days.
1,8601,"Tennyson, Alfred Tennyson, Baron, 1809-1892",The Early Poems of Alfred Lord Tennyson,"Produced by Jonathan Ingram, Clytie Siddall, C...","""The Early Poems of Alfred Lord Tennyson"" by J...",English,PR: Language and Literatures: English literature,English poetry -- 19th century,NaN,NaN,NaN,Text,8601,"Aug 1, 2005","Jun 21, 2020",Public domain in the USA.,603 downloads in the last 30 days.
2,8602,"Riddell, J. H., Mrs., 1832-1906",The Uninhabited House,"Produced by Suzanne Shell, Martin Agren, Charl...","""The Uninhabited House"" by Mrs. J. H. Riddell ...",English,PR: Language and Literatures: English literature,Ghost stories,English fiction -- 19th century,NaN,NaN,Text,8602,"Aug 1, 2005","Apr 10, 2014",Public domain in the USA.,90 downloads in the last 30 days.
3,8603,Various,"Notes and Queries, Number 01, November 3, 1849",Produced by Internet Library of Early Journals...,"""Notes and Queries, Number 01, November 3, 184...",English,AG: General Works: Dictionaries and other gene...,Questions and answers -- Periodicals,NaN,NaN,NaN,Text,8603,"Aug 1, 2005","Jan 17, 2013",Public domain in the USA.,50 downloads in the last 30 days.
4,8604,"Aeschylus, 526 BCE-457 BCE","The House of Atreus; Being the Agamemnon, the ...","Ted Garvin, Lorna Hanrahan, Charles Franks, an...","""The House of Atreus; Being the Agamemnon, the...",English,PA: Language and Literatures: Classical Langua...,"Agamemnon, King of Mycenae (Mythological chara...","Orestes, King of Argos (Mythological character...",NaN,NaN,Text,8604,"Aug 1, 2005","Jun 19, 2023",Public domain in the USA.,210 downloads in the last 30 days.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19924,19996,Various,"The Atlantic Monthly, Volume 16, No. 96, Octob...","Produced by Joshua Hutchinson, Josephine Paolu...","""The Atlantic Monthly, Volume 16, No. 96, Octo...",English,AP: General Works: Periodicals,American periodicals,NaN,NaN,NaN,Text,19996,"Dec 2, 2006",NaN,Public domain in the USA.,88 downloads in the last 30 days.
19925,19997,"Ober, Frederick A. (Frederick Albion), 1849-1913",Amerigo Vespucci,"Produced by Suzanne Shell, Richard J. Shiffer ...","""Amerigo Vespucci"" by Frederick A. Ober is a h...",English,E011: History: America: America,"Vespucci, Amerigo, 1451-1512",NaN,NaN,NaN,Text,19997,"Dec 3, 2006",NaN,Public domain in the USA.,145 downloads in the last 30 days.
19926,19998,"Allen, Lewis Falley, 1800-1890",Rural ArchitectureBeing a Complete Description...,"Produced by Louise Hope, Steven Giacomelli and...","""Rural Architecture"" by Lewis Falley Allen is ...",English,NA: Fine Arts: Architecture,Domestic animals,"Architecture, Domestic",NaN,NaN,Text,19998,"Dec 3, 2006",NaN,Public domain in the USA.,232 downloads in the last 30 days.
19927,19999,"Trowbridge, J. T. (John Townsend), 1827-1916",The Drummer Boy,Produced by David Edwards and the Online Distr...,"""The Drummer Boy"" by J. T. Trowbridge is a his...",English,PZ: Language and Literatures: Juvenile belles ...,"United States -- History -- Civil War, 1861-18...",NaN,NaN,NaN,Text,19999,"Dec 3, 2006",NaN,Public domain in the USA.,76 downloads in the last 30 days.


In [5]:
# Sélection des colonnes souhaitées
colonnes_souhaitees = ['Ebook ID', 'Author', 'Title', 'Credits', 'Summary', 'Language', 'LoC Class', 'Subject']
df_filtre = df[colonnes_souhaitees]

In [6]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import requests

# Initialiser le modèle ChatOpenAI
chat_model = ChatOpenAI(model="gpt-3.5-turbo", 
                         openai_api_key=azure_openai_api_key,
                         engine=deployment_name)

C:\Users\antoa\AppData\Local\Temp\ipykernel_18928\2150038817.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-3.5-turbo",
WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [8]:
# Fonction pour répondre aux questions
def ask_book_question(question, book_attributes):
    # Concaténer les attributs du livre pour fournir le contexte
    context = "\n".join(book_attributes)
    
    # Créer le message pour le modèle
    messages = [
        HumanMessage(content=f"{context}\n\nQuestion: {question}")
    ]
    
    # Utiliser le modèle pour générer la réponse
    response = chat_model(messages)
    return response.content

# Fonction pour obtenir les informations d'un livre
def get_book_info(book_title):
    # Filtrer les informations du livre correspondant au titre donné
    book_info = df_filtre[df_filtre['Title'].str.contains(book_title, case=False, na=False)]

    # Vérifier si le livre existe dans le DataFrame
    if not book_info.empty:
        # Attributs du livre à utiliser comme contexte
        book_attributes = [
            f"Titre: {book_info.iloc[0]['Title']}",
            f"Auteur: {book_info.iloc[0]['Author']}",
            f"Langue: {book_info.iloc[0]['Language']}",
            f"Sujet: {book_info.iloc[0]['Subject']}",
            f"Résumé: {book_info.iloc[0]['Summary']}",
        ]
        return book_attributes, book_info.iloc[0]['Title'], book_info.iloc[0]['Ebook ID']
    else:
        return None, None, None

# Fonction pour récupérer le texte d'un livre à partir de Gutenberg
def fetch_book_text(ebook_id):
    url = f"https://www.gutenberg.org/ebooks/{ebook_id}.txt.utf-8"  # URL pour le téléchargement en .txt
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Demander à l'utilisateur de saisir le titre du livre
book_title = input("Veuillez entrer le titre du livre : ")

# Obtenir les attributs du livre
book_attributes, title, ebook_id = get_book_info(book_title)

if book_attributes is not None:
    # Poser une question sur le sujet du livre
    question_subject = f"De quels sujets traite le livre '{title}' ?"
    subjects = ask_book_question(question_subject, book_attributes)
    print(f"Sujets traités dans le livre : {subjects}")

    # Poser une question pour extraire les noms des personnages
    question_characters = f"Quels sont les noms des personnages mentionnés dans le résumé du livre '{title}' ?"
    character_names = ask_book_question(question_characters, book_attributes)
    print(f"Noms des personnages mentionnés dans le résumé : {character_names}")

    # Demander à l'utilisateur s'il souhaite récupérer le texte du livre
    fetch_text = input("Souhaitez-vous récupérer le texte complet du livre ? (oui/non) : ")
    
    if fetch_text.lower() == 'oui':
        book_text = fetch_book_text(ebook_id)
        if book_text:
            print("Texte du livre récupéré avec succès.")
            # Poser une question sur le texte du livre
            question_text = input("Posez votre question sur le texte du livre : ")
            answer_text = ask_book_question(question_text, [book_text])
            print(f"Réponse : {answer_text}")
        else:
            print("Erreur lors de la récupération du texte du livre.")
else:
    print("Aucun livre trouvé avec ce titre.")

Aucun livre trouvé avec ce titre.
